Adatok előfeldolgozása




In [11]:
import requests
import zipfile
import os
import json
from collections import Counter
import re
import pandas as pd

In [3]:


url = "https://bmeedu-my.sharepoint.com/:u:/g/personal/gyires-toth_balint_vik_bme_hu/IQDYwXUJcB_jQYr0bDfNT5RKARYgfKoH97zho3rxZ46KA1I?e=iFp3iz&download=1"

response = requests.get(url)

print(response.status_code)        # should be 200
print(response.headers.get('content-type'))  # should be 'application/zip' or similar

# Optional: save a small part to check
with open("/content/legaltextdecoder.zip", "wb") as f:
    f.write(response.content)  # first 1 KB
print("Downloaded 1 KB to /content/legaltextdecoder.zip")

200
application/x-zip-compressed
Downloaded 1 KB to /content/legaltextdecoder.zip


In [4]:
zip_path = "legaltextdecoder.zip"
extract_path = "data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [5]:
base_dir = "data"
all_data = []

json_file_count = 0
record_count = 0

for root, dirs, files in os.walk(base_dir):
    dirs[:] = [d for d in dirs if d != "consensus"]

    for file in files:
        if file.endswith(".json"):
            json_file_count += 1
            file_path = os.path.join(root, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

                if isinstance(data, list):
                    all_data.extend(data)
                    record_count += len(data)
                else:
                    all_data.append(data)
                    record_count += 1

print(f"JSON files loaded: {json_file_count}")
print(f"Total data records: {record_count}")

JSON files loaded: 34
Total data records: 3897


In [6]:
texts = []
labels = []

for item in all_data:
    text = item.get("data", {}).get("text")

    if not text:
        continue

    annotations = item.get("annotations", [])
    if not annotations:
        continue

    ann = annotations[0]
    results = ann.get("result", [])
    if not results:
        continue

    choices = results[0].get("value", {}).get("choices", [])
    if not choices:
        continue

    label = choices[0]

    texts.append(text)
    labels.append(label)

print("Loaded data records:", len(texts))

Loaded data records: 3747


In [7]:
label_counts = Counter(labels)

print("Label distribution:")
for label, count in label_counts.items():
    print(f"{label}: {count}")


Label distribution:
4-Érthető: 1122
3-Többé/kevésbé megértem: 816
5-Könnyen érthető: 1159
2-Nehezen érthető: 450
1-Nagyon nehezen érthető: 200


In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

clean_texts = [clean_text(t) for t in texts]

df = pd.DataFrame({
    "text": clean_texts,
    "label": labels
})

print(f"Samples after cleaning: {len(df)}")

duplicate_count = df.duplicated(subset="text").sum()
print(f"Duplicate texts detected: {duplicate_count}")

df = df.drop_duplicates(subset="text")
print(f"Samples after removing duplicates: {len(df)}")

print("Label distribution:")
print(f"\n{df['label'].value_counts()}")

df["text_length"] = df["text"].apply(lambda x: len(x.split()))

print("Text length statistics:")
print("\n" + df["text_length"].describe().to_string())

label_counts = df["label"].value_counts()
imbalance_ratio = label_counts.max() / label_counts.min()
print("Class imbalance analysis:")
print(f"\n{label_counts}")
print(f"Imbalance ratio (max/min): {imbalance_ratio:.2f}")

short_texts = (df["text_length"] < 5).sum()
print(f"Texts with fewer than 5 words: {short_texts}")

Samples after cleaning: 3747
Duplicate texts detected: 354
Samples after removing duplicates: 3393
Label distribution:

label
5-Könnyen érthető           1094
4-Érthető                   1021
3-Többé/kevésbé megértem     722
2-Nehezen érthető            389
1-Nagyon nehezen érthető     167
Name: count, dtype: int64
Text length statistics:

count    3393.000000
mean       50.390510
std        56.480726
min         1.000000
25%        21.000000
50%        38.000000
75%        62.000000
max      1186.000000
Class imbalance analysis:

label
5-Könnyen érthető           1094
4-Érthető                   1021
3-Többé/kevésbé megértem     722
2-Nehezen érthető            389
1-Nagyon nehezen érthető     167
Name: count, dtype: int64
Imbalance ratio (max/min): 6.55
Texts with fewer than 5 words: 93


In [16]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
#Baseline modell

#Túltanítás
train_texts = texts[:8]
train_labels = labels[:8]

test_texts = texts[8:]
test_labels = labels[8:]

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train = label_encoder.transform(train_labels)
y_test = label_encoder.transform(test_labels)

NUM_CLASSES = 5
y_train = np.clip(y_train, 0, NUM_CLASSES - 1)
y_test = np.clip(y_test, 0, NUM_CLASSES - 1)

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)

MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train = tfidf.fit_transform(train_texts).toarray()
X_test = tfidf.transform(test_texts).toarray()

print("TF-IDF feature size:", X_train.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=2,
    verbose=1
)


TF-IDF feature size: 86


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 32)             │         2,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,005 (15.64 KB)

 Trainable params: 4,005 (15.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3167 - loss: 1.5985
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2333 - loss: 1.5454    
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9500 - loss: 1.4953 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7667 - loss: 1.4741 
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9500 - loss: 1.3850 
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8667 - loss: 1.3597
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8667 - loss: 1.3214 
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9500 - loss: 1.2699 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8667 - loss: 1.1997 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7667 - loss: 1.2131
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9500 - loss: 1.0694 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9500 

Az overfitt megtörtént.

Baseline modell betanítása

In [14]:
from sklearn.model_selection import train_test_split

# Train vs temp (val + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    texts,
    labels,
    test_size=0.25,
    stratify=labels,
    random_state=42
)

# Val vs test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.40,
    stratify=y_temp,
    random_state=42
)

print("Train size:", len(X_train))
print("Val size:  ", len(X_val))
print("Test size: ", len(X_test))


Train size: 2810
Val size:   562
Test size:  375


In [17]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)  # Fit on all labels

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

# One-hot encoding for MLP
NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# TF-IDF Vectorization
# -----------------------------
MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train_vec = tfidf.fit_transform(X_train).toarray()
X_val_vec   = tfidf.transform(X_val).toarray()
X_test_vec  = tfidf.transform(X_test).toarray()

print("TF-IDF feature size:", X_train_vec.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_vec.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Hyperparameters:\nEpochs: 20\nBatch size: 16\nLearning rate: {model.optimizer.learning_rate.numpy()}")
model.summary()

# -----------------------------
# Train (with validation)
# -----------------------------
model.fit(
    X_train_vec,
    y_train_enc,
    validation_data=(X_val_vec, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_vec)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

TF-IDF feature size: 2000
Hyperparameters:
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │        64,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,253 (254.89 KB)

 Trainable params: 65,253 (254.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.2844 - loss: 1.5587 - val_accuracy: 0.3932 - val_loss: 1.4119
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4470 - loss: 1.3411 - val_accuracy: 0.4484 - val_loss: 1.2863
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5852 - loss: 1.0885 - val_accuracy: 0.4609 - val_loss: 1.2836
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6986 - loss: 0.8571 - val_accuracy: 0.4555 - val_loss: 1.3235
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7899 - loss: 0.6526 - val_accuracy: 0.4502 - val_loss: 1.4295
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8514 - loss: 0.5148 - val_accuracy: 0.4573 - val_loss: 1.5925
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8982 - loss: 0.3767 - val_accuracy: 0.4466 - val_loss: 1.7197
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9242 - loss: 0.2803 - val_accuracy: 0.

Early stopping hozzáadása


In [18]:
from tensorflow.keras.callbacks import EarlyStopping


# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)  # Fit on all labels

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# TF-IDF Vectorization
# -----------------------------
MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train_vec = tfidf.fit_transform(X_train).toarray()
X_val_vec   = tfidf.transform(X_val).toarray()
X_test_vec  = tfidf.transform(X_test).toarray()

print("TF-IDF feature size:", X_train_vec.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_vec.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(f"Hyperparameters:\nEpochs: 100\nBatch size: 16\nLearning rate: {model.optimizer.learning_rate.numpy()}")
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',   # Stop when validation loss stops improving
    patience=5,           # Wait 5 epochs before stopping
    restore_best_weights=True
)

# -----------------------------
# Train with EarlyStopping
# -----------------------------
model.fit(
    X_train_vec,
    y_train_enc,
    validation_data=(X_val_vec, y_val_enc),
    epochs=100,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_vec)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

TF-IDF feature size: 2000
Hyperparameters:
Epochs: 100
Batch size: 16
Learning rate: 0.0010000000474974513


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │        64,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,253 (254.89 KB)

 Trainable params: 65,253 (254.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3019 - loss: 1.5410 - val_accuracy: 0.3808 - val_loss: 1.4134
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4558 - loss: 1.3204 - val_accuracy: 0.4270 - val_loss: 1.2844
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5699 - loss: 1.0815 - val_accuracy: 0.4609 - val_loss: 1.2583
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6878 - loss: 0.8506 - val_accuracy: 0.4520 - val_loss: 1.3151
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7990 - loss: 0.6435 - val_accuracy: 0.4520 - val_loss: 1.4087
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8666 - loss: 0.4843 - val_accuracy: 0.4413 - val_loss: 1.5354
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9012 - loss: 0.3682 - val_accuracy: 0.4448 - val_loss: 1.6998
Epoch 8/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9353 - loss: 0.2598 - val_acc

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TFIDF csere Embeddingre

In [20]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100
print(f"Encoder info: num_words: {MAX_VOCAB}\n max_length: {MAX_LEN}")

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(f"Hyperparameters:\nEpochs: 20\nBatch size: 16\nLearning rate: {model.optimizer.learning_rate.numpy()}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

Encoder info: num_words: 5000
 max_length: 100
Hyperparameters:
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

 Trainable params: 263,653 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.3763 - loss: 1.4165 - val_accuracy: 0.4466 - val_loss: 1.3044
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5365 - loss: 1.1967 - val_accuracy: 0.4698 - val_loss: 1.2603
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7536 - loss: 0.7585 - val_accuracy: 0.4662 - val_loss: 1.3624
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8993 - loss: 0.3483 - val_accuracy: 0.4431 - val_loss: 1.5703
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9407 - loss: 0.2095 - val_accuracy: 0.4377 - val_loss: 1.7571
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9476 - loss: 0.1516 - val_accuracy: 0.4324 - val_loss: 1.8484
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9551 - loss: 0.1337 - val_accuracy: 0.4395 - val_loss: 1.9760
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          precis

LSTM kipróbálása

In [22]:
from tensorflow.keras.layers import LSTM
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → LSTM → Dense)
# -----------------------------
EMBED_DIM = 32
LSTM_UNITS = 16

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    LSTM(LSTM_UNITS),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"LSTM units: {LSTM_UNITS}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
LSTM units: 16
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 163,845 (640.02 KB)

 Trainable params: 163,845 (640.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.2918 - loss: 1.5050 - val_accuracy: 0.3612 - val_loss: 1.3648
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.3557 - loss: 1.3920 - val_accuracy: 0.3826 - val_loss: 1.3437
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 17s 97ms/step - accuracy: 0.3998 - loss: 1.3443 - val_accuracy: 0.4181 - val_loss: 1.3115
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - accuracy: 0.4496 - loss: 1.2817 - val_accuracy: 0.4395 - val_loss: 1.2789
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - accuracy: 0.5222 - loss: 1.1558 - val_accuracy: 0.4822 - val_loss: 1.2330
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step - accuracy: 0.5740 - loss: 1.0671 - val_accuracy: 0.4680 - val_loss: 1.2839
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 16s 92ms/step - accuracy: 0.6384 - loss: 0.9652 - val_accuracy: 0.4822 - val_loss: 1.2892
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.6770 - loss: 0.8926 - v

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GRU kipróbálása (elvileg jó kis adathalmazokon)

In [23]:
from tensorflow.keras.layers import GRU
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → LSTM → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(32),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,557 (654.52 KB)

 Trainable params: 167,557 (654.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 25s 121ms/step - accuracy: 0.3099 - loss: 1.4993 - val_accuracy: 0.3221 - val_loss: 1.4191
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 28s 50ms/step - accuracy: 0.3626 - loss: 1.4122 - val_accuracy: 0.4288 - val_loss: 1.3379
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.4400 - loss: 1.3035 - val_accuracy: 0.4502 - val_loss: 1.2443
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.5723 - loss: 1.0582 - val_accuracy: 0.4413 - val_loss: 1.2680
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.6216 - loss: 0.9610 - val_accuracy: 0.4555 - val_loss: 1.3576
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.6842 - loss: 0.8351 - val_accuracy: 0.4359 - val_loss: 1.4656
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.7398 - loss: 0.7122 - val_accuracy: 0.3861 - val_loss: 1.8070
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.7807 - loss: 0.6440 - 

Dropout bevezetése

In [24]:
from tensorflow.keras.layers import Dropout

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"GRU units: {GRU_UNITS}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
GRU units: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,557 (654.52 KB)

 Trainable params: 167,557 (654.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.2987 - loss: 1.5121 - val_accuracy: 0.3345 - val_loss: 1.4260
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.3283 - loss: 1.4328 - val_accuracy: 0.3452 - val_loss: 1.4053
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - accuracy: 0.3494 - loss: 1.3963 - val_accuracy: 0.4466 - val_loss: 1.2759
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.4760 - loss: 1.2482 - val_accuracy: 0.4075 - val_loss: 1.2796
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.5438 - loss: 1.1018 - val_accuracy: 0.4466 - val_loss: 1.2638
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.5933 - loss: 0.9925 - val_accuracy: 0.4377 - val_loss: 1.3444
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - accuracy: 0.6733 - loss: 0.8559 - val_accuracy: 0.4413 - val_loss: 1.4309
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.7148 - loss: 0.7998 - 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Regularizáció bevezetése

In [25]:
from tensorflow.keras.regularizers import l2

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout
L2_REG = 1e-4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"GRU units: {GRU_UNITS}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
print(f"L2 regularization: {L2_REG}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
GRU units: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.3
L2 regularization: 0.0001


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,557 (654.52 KB)

 Trainable params: 167,557 (654.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 32s 153ms/step - accuracy: 0.2823 - loss: 1.5383 - val_accuracy: 0.3470 - val_loss: 1.4246
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 17s 94ms/step - accuracy: 0.3041 - loss: 1.4674 - val_accuracy: 0.3719 - val_loss: 1.3668
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 16s 89ms/step - accuracy: 0.3471 - loss: 1.3950 - val_accuracy: 0.4395 - val_loss: 1.3026
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 23s 101ms/step - accuracy: 0.4532 - loss: 1.2711 - val_accuracy: 0.4324 - val_loss: 1.2929
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.5520 - loss: 1.1060 - val_accuracy: 0.4377 - val_loss: 1.2731
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.6031 - loss: 1.0102 - val_accuracy: 0.4537 - val_loss: 1.3599
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 20s 104ms/step - accuracy: 0.6823 - loss: 0.8700 - val_accuracy: 0.4786 - val_loss: 1.4342
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.7246 - loss: 0.781

In [26]:
from tensorflow.keras.layers import Dropout

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"GRU units: {GRU_UNITS}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
GRU units: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 327,557 (1.25 MB)

 Trainable params: 327,557 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - accuracy: 0.2911 - loss: 1.5232 - val_accuracy: 0.3185 - val_loss: 1.4296
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - accuracy: 0.3436 - loss: 1.4389 - val_accuracy: 0.3683 - val_loss: 1.3637
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.3782 - loss: 1.3639 - val_accuracy: 0.4270 - val_loss: 1.2875
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.5197 - loss: 1.1718 - val_accuracy: 0.4573 - val_loss: 1.3345
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.5911 - loss: 1.0146 - val_accuracy: 0.4395 - val_loss: 1.4202
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 121ms/step - accuracy: 0.6868 - loss: 0.8579 - val_accuracy: 0.4680 - val_loss: 1.5410
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 39s 113ms/step - accuracy: 0.7638 - loss: 0.7102 - val_accuracy: 0.4484 - val_loss: 1.7189
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7644 - loss: 0.67

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
from tensorflow.keras.regularizers import l2

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout
L2_REG = 1e-4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"GRU units: {GRU_UNITS}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
print(f"L2 regularization: {L2_REG}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
GRU units: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.3
L2 regularization: 0.0001


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 327,557 (1.25 MB)

 Trainable params: 327,557 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 29s 132ms/step - accuracy: 0.3020 - loss: 1.5362 - val_accuracy: 0.3470 - val_loss: 1.4416
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.3021 - loss: 1.4648 - val_accuracy: 0.3470 - val_loss: 1.4030
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step - accuracy: 0.3295 - loss: 1.4093 - val_accuracy: 0.4324 - val_loss: 1.3113
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.4597 - loss: 1.2451 - val_accuracy: 0.4324 - val_loss: 1.2541
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 31s 121ms/step - accuracy: 0.5577 - loss: 1.0883 - val_accuracy: 0.4484 - val_loss: 1.2709
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 38s 105ms/step - accuracy: 0.6576 - loss: 0.9279 - val_accuracy: 0.4555 - val_loss: 1.3635
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step - accuracy: 0.7034 - loss: 0.8347 - val_accuracy: 0.4662 - val_loss: 1.5035
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.7626 - loss: 0.680

Visszatértem sima neurális hálókhoz, az LSTM és a GRU nem teljesítettek jól a tesztjeim alatt.

Különböző perceptron számok és több layer tesztelése


In [28]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 367,109 (1.40 MB)

 Trainable params: 367,109 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3987 - loss: 1.3998 - val_accuracy: 0.4413 - val_loss: 1.2945
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5390 - loss: 1.1522 - val_accuracy: 0.4875 - val_loss: 1.2316
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7910 - loss: 0.6844 - val_accuracy: 0.4769 - val_loss: 1.3687
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9249 - loss: 0.2909 - val_accuracy: 0.4520 - val_loss: 1.6247
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9495 - loss: 0.1646 - val_accuracy: 0.4164 - val_loss: 1.7422
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9629 - loss: 0.1255 - val_accuracy: 0.4288 - val_loss: 1.7879
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9647 - loss: 0.1058 - val_accuracy: 0.4431 - val_loss: 1.8588
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

Test Classification Report:
                          prec

In [29]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 368,165 (1.40 MB)

 Trainable params: 368,165 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3922 - loss: 1.3978 - val_accuracy: 0.4466 - val_loss: 1.2913
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5230 - loss: 1.1796 - val_accuracy: 0.4626 - val_loss: 1.2776
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7564 - loss: 0.6916 - val_accuracy: 0.4573 - val_loss: 1.4576
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9190 - loss: 0.2871 - val_accuracy: 0.4769 - val_loss: 1.6948
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9458 - loss: 0.1764 - val_accuracy: 0.4431 - val_loss: 1.8357
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9595 - loss: 0.1376 - val_accuracy: 0.4626 - val_loss: 2.0276
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9602 - loss: 0.1129 - val_accuracy: 0.4466 - val_loss: 2.0569
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          precisi

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Innentől rengeteg kombinációban kipróbáltam különböző hálókat különböző dropout paraméterekkel, regularizácioval, embedding mérettel és vocabulary mérettel


In [30]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.2
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(64, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(32, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(32, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  print("\n===== TRAINING CONFIGURATION =====")
  print(f"MAX_VOCAB: {MAX_VOCAB}")
  print(f"MAX_LEN: {MAX_LEN}")
  print(f"Embedding dim: {EMBED_DIM}")
  print(f"Epochs: 20")
  print(f"Batch size: 16")
  print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
  print(f"Number of classes: {NUM_CLASSES}")
  print(f"Dropout rate: {DROPOUT_RATE}")
  model.build(input_shape=(None, MAX_LEN))
  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.2


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 368,165 (1.40 MB)

...

 Total params: 368,165 (1.40 MB)

 Trainable params: 368,165 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.3136 - loss: 1.4771 - val_accuracy: 0.4164 - val_loss: 1.4205
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3437 - loss: 1.4004 - val_accuracy: 0.4359 - val_loss: 1.3074
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4608 - loss: 1.2638 - val_accuracy: 0.4609 - val_loss: 1.2654
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5612 - loss: 1.0893 - val_accuracy: 0.3968 - val_loss: 1.4065
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6966 - loss: 0.8371 - val_accuracy: 0.4484 - val_loss: 1.3651
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8009 - loss: 0.5727 - val_accuracy: 0.4217 - val_loss: 1.4551
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8465 - loss: 0.4527 - val_accuracy: 0.4698 - val_loss: 1.8590
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8790 - loss: 0.3554 - val_accu

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Bag of Word kipróbálása

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
MAX_FEATURES = 5000
vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

X_train_bow = vectorizer.fit_transform(X_train).toarray()
X_val_bow   = vectorizer.transform(X_val).toarray()
X_test_bow  = vectorizer.transform(X_test).toarray()

print("BoW feature size:", X_train_bow.shape[1])

# -----------------------------
# Neural Network (Dense + Dropout)
# -----------------------------
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\n===== TRAINING CONFIGURATION =====")
    print(f"Max Features: {MAX_FEATURES}")
    print(f"Embedding dim: {EMBED_DIM}")
    print(f"Epochs: 20")
    print(f"Batch size: 16")
    print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
    print(f"Number of classes: {NUM_CLASSES}")
    print(f"Dropout rate: {DROPOUT_RATE}")
    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_bow,
        y_train_enc,
        validation_data=(X_val_bow, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_bow)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report (Dropout={:.1f}):".format(DROPOUT_RATE))
    print(classification_report(y_test, decoded_preds))



BoW feature size: 5000

===== TRAINING CONFIGURATION =====
Max Features: 5000
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.2


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_49 (Dense)                │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_57 (Dense)                │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

 Trainable params: 323,365 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.2722 - loss: 1.5841 - val_accuracy: 0.3025 - val_loss: 1.4898
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3555 - loss: 1.4827 - val_accuracy: 0.4004 - val_loss: 1.3780
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3913 - loss: 1.3665 - val_accuracy: 0.4253 - val_loss: 1.2821
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4810 - loss: 1.2325 - val_accuracy: 0.4573 - val_loss: 1.2382
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5624 - loss: 1.0860 - val_accuracy: 0.4502 - val_loss: 1.2386
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6192 - loss: 0.9835 - val_accuracy: 0.4466 - val_loss: 1.2748
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6638 - loss: 0.8592 - val_accuracy: 0.4555 - val_loss: 1.3187
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7324 - loss: 0.7615 - val_accuracy:

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
MAX_FEATURES = 10000
vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

X_train_bow = vectorizer.fit_transform(X_train).toarray()
X_val_bow   = vectorizer.transform(X_val).toarray()
X_test_bow  = vectorizer.transform(X_test).toarray()

print("BoW feature size:", X_train_bow.shape[1])

# -----------------------------
# Neural Network (Dense + Dropout)
# -----------------------------
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\n===== TRAINING CONFIGURATION =====")
    print(f"Max Features: {MAX_FEATURES}")
    print(f"Embedding dim: {EMBED_DIM}")
    print(f"Epochs: 20")
    print(f"Batch size: 16")
    print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
    print(f"Number of classes: {NUM_CLASSES}")
    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_bow,
        y_train_enc,
        validation_data=(X_val_bow, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_bow)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report (Dropout={:.1f}):".format(DROPOUT_RATE))
    print(classification_report(y_test, decoded_preds))


BoW feature size: 10000

===== TRAINING CONFIGURATION =====
Max Features: 10000
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_61 (Dense)                │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,365 (2.45 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_69 (Dense)                │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,365 (2.45 MB)

 Trainable params: 643,365 (2.45 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.2773 - loss: 1.5910 - val_accuracy: 0.3737 - val_loss: 1.4770
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.3559 - loss: 1.4677 - val_accuracy: 0.4004 - val_loss: 1.3798
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.4336 - loss: 1.3677 - val_accuracy: 0.4075 - val_loss: 1.3082
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.5252 - loss: 1.2138 - val_accuracy: 0.4359 - val_loss: 1.2708
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.6015 - loss: 1.0426 - val_accuracy: 0.4520 - val_loss: 1.2647
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6811 - loss: 0.8928 - val_accuracy: 0.4537 - val_loss: 1.3182
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7297 - loss: 0.7609 - val_accuracy: 0.4377 - val_loss: 1.4055
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.7717 - loss: 0.6679 - val_accu

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
for MAX_FEATURES in [5000, 10000, 20000]:
  vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

  X_train_bow = vectorizer.fit_transform(X_train).toarray()
  X_val_bow   = vectorizer.transform(X_val).toarray()
  X_test_bow  = vectorizer.transform(X_test).toarray()

  print("BoW feature size:", X_train_bow.shape[1])

  # -----------------------------
  # Neural Network (Dense, no Dropout)
  # -----------------------------
  model = Sequential([
      Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )
  print("\n===== TRAINING CONFIGURATION =====")
  print(f"Max Features: {MAX_FEATURES}")
  print(f"Embedding dim: {EMBED_DIM}")
  print(f"Epochs: 20")
  print(f"Batch size: 16")
  print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
  print(f"Number of classes: {NUM_CLASSES}")
  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_bow,
      y_train_enc,
      validation_data=(X_val_bow, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_bow)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))


BoW feature size: 5000

===== TRAINING CONFIGURATION =====
Max Features: 5000
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_73 (Dense)                │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_81 (Dense)                │ (None, 64)             │     1,280,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283,365 (4.90 MB)

 Trainable params: 1,283,365 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.3138 - loss: 1.5233 - val_accuracy: 0.4395 - val_loss: 1.2850
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.6843 - loss: 0.8965 - val_accuracy: 0.4733 - val_loss: 1.2860
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8934 - loss: 0.3883 - val_accuracy: 0.4662 - val_loss: 1.6112
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.9367 - loss: 0.2307 - val_accuracy: 0.4893 - val_loss: 1.7728
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9356 - loss: 0.1544 - val_accuracy: 0.4804 - val_loss: 1.9702
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9455 - loss: 0.1313 - val_accuracy: 0.4448 - val_loss: 1.9681
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Test Classification Report:
                          precision    recall  f1-score   support

1-Nagyon nehezen érthető       0.40      0.10      0.16        20
       2-Nehezen érthető

Visszatérés embeddingre

In [35]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:

  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(64, activation='relu'),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  print("\n===== TRAINING CONFIGURATION =====")
  print(f"MAX_VOCAB: {MAX_VOCAB}")
  print(f"MAX_LEN: {MAX_LEN}")
  print(f"Embedding dim: {EMBED_DIM}")
  print(f"Epochs: 20")
  print(f"Batch size: 16")
  print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
  print(f"Number of classes: {NUM_CLASSES}")
  model.build(input_shape=(None, MAX_LEN))
  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 368,165 (1.40 MB)

...

 Total params: 1,462,565 (5.58 MB)

 Trainable params: 1,462,565 (5.58 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.3856 - loss: 1.3956 - val_accuracy: 0.4431 - val_loss: 1.2828
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.5726 - loss: 1.1092 - val_accuracy: 0.4573 - val_loss: 1.2881
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8468 - loss: 0.4811 - val_accuracy: 0.4359 - val_loss: 1.4822
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9445 - loss: 0.1912 - val_accuracy: 0.3950 - val_loss: 1.9215
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9601 - loss: 0.1276 - val_accuracy: 0.4680 - val_loss: 1.7893
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9601 - loss: 0.1023 - val_accuracy: 0.4573 - val_loss: 1.9939
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Test Classification Report:
                          precision    recall  f1-score   support

1-Nagyon nehezen érthető       0.00      0.00      0.00        20
       2-Nehezen érthető

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:

  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  print("\n===== TRAINING CONFIGURATION =====")
  print(f"MAX_VOCAB: {MAX_VOCAB}")
  print(f"MAX_LEN: {MAX_LEN}")
  print(f"Embedding dim: {EMBED_DIM}")
  print(f"Epochs: 20")
  print(f"Batch size: 16")
  print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
  print(f"Number of classes: {NUM_CLASSES}")
  model.build(input_shape=(None, MAX_LEN))
  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_97 (Dense)                │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)        │ (None, 100, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 32)             │       409,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,050,853 (4.01 MB)

 Trainable params: 1,050,853 (4.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.3728 - loss: 1.4101 - val_accuracy: 0.4217 - val_loss: 1.2928
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6070 - loss: 1.0620 - val_accuracy: 0.4929 - val_loss: 1.2470
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8592 - loss: 0.4744 - val_accuracy: 0.4484 - val_loss: 1.4762
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9356 - loss: 0.2162 - val_accuracy: 0.4626 - val_loss: 1.5804
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9597 - loss: 0.1310 - val_accuracy: 0.4662 - val_loss: 1.6734
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9692 - loss: 0.0941 - val_accuracy: 0.4520 - val_loss: 1.8112
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9657 - loss: 0.0846 - val_accuracy: 0.4573 - val_loss: 1.8448
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          pre

In [37]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:
    for DROPOUT_RATE in [0.2, 0.3, 0.4]:

      model = Sequential([
          Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
          Flatten(),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(NUM_CLASSES, activation='softmax')
      ])

      model.compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy']
      )

      print("\n===== TRAINING CONFIGURATION =====")
      print(f"MAX_VOCAB: {MAX_VOCAB}")
      print(f"MAX_LEN: {MAX_LEN}")
      print(f"Embedding dim: {EMBED_DIM}")
      print(f"Epochs: 20")
      print(f"Batch size: 16")
      print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
      print(f"Number of classes: {NUM_CLASSES}")
      print(f"Dropout rate: {DROPOUT_RATE}")
      model.build(input_shape=(None, MAX_LEN))
      model.summary()

      # -----------------------------
      # EarlyStopping callback
      # -----------------------------
      early_stop = EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      )

      # -----------------------------
      # Train
      # -----------------------------
      model.fit(
          X_train_pad,
          y_train_enc,
          validation_data=(X_val_pad, y_val_enc),
          epochs=20,
          batch_size=16,
          verbose=1,
          callbacks=[early_stop]
      )

      # -----------------------------
      # Evaluate on test set
      # -----------------------------
      test_preds = model.predict(X_test_pad)
      test_preds = np.argmax(test_preds, axis=1)
      decoded_preds = label_encoder.inverse_transform(test_preds)

      print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
      print(classification_report(y_test, decoded_preds))



===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.2


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_20 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_28 (Embedding)        │ (None, 100, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_130 (Dense)               │ (None, 32)             │       409,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_131 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_132 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,050,853 (4.01 MB)

 Trainable params: 1,050,853 (4.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.3408 - loss: 1.5009 - val_accuracy: 0.4377 - val_loss: 1.3259
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.4406 - loss: 1.3532 - val_accuracy: 0.4484 - val_loss: 1.2657
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.5443 - loss: 1.1393 - val_accuracy: 0.4591 - val_loss: 1.2380
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7168 - loss: 0.7885 - val_accuracy: 0.4626 - val_loss: 1.3012
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8499 - loss: 0.4763 - val_accuracy: 0.4164 - val_loss: 1.4801
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8856 - loss: 0.3736 - val_accuracy: 0.4609 - val_loss: 1.6198
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9269 - loss: 0.2575 - val_accuracy: 0.4057 - val_loss: 1.8113
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9414 - loss: 0.2058 - val_accu

In [38]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
for EMBED_DIM in [32]:
    for DROPOUT_RATE in [0.0, 0.3]:

      model = Sequential([
          Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
          Flatten(),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(NUM_CLASSES, activation='softmax')
      ])

      model.compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy']
      )

      print("\n===== TRAINING CONFIGURATION =====")
      print(f"MAX_VOCAB: {MAX_VOCAB}")
      print(f"MAX_LEN: {MAX_LEN}")
      print(f"Embedding dim: {EMBED_DIM}")
      print(f"Epochs: 20")
      print(f"Batch size: 16")
      print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
      print(f"Number of classes: {NUM_CLASSES}")
      print(f"Dropout rate: {DROPOUT_RATE}")
      model.build(input_shape=(None, MAX_LEN))
      model.summary()

      # -----------------------------
      # EarlyStopping callback
      # -----------------------------
      early_stop = EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      )

      # -----------------------------
      # Train
      # -----------------------------
      model.fit(
          X_train_pad,
          y_train_enc,
          validation_data=(X_val_pad, y_val_enc),
          epochs=20,
          batch_size=16,
          verbose=1,
          callbacks=[early_stop]
      )

      # -----------------------------
      # Evaluate on test set
      # -----------------------------
      test_preds = model.predict(X_test_pad)
      test_preds = np.argmax(test_preds, axis=1)
      decoded_preds = label_encoder.inverse_transform(test_preds)

      print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
      print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.0


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_29 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_133 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_52 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_134 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_53 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_135 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_30 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_136 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_137 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_138 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

 Trainable params: 263,653 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.3284 - loss: 1.4869 - val_accuracy: 0.4235 - val_loss: 1.3241
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4358 - loss: 1.3069 - val_accuracy: 0.4484 - val_loss: 1.2703
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5661 - loss: 1.1024 - val_accuracy: 0.4698 - val_loss: 1.2426
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6990 - loss: 0.8034 - val_accuracy: 0.4840 - val_loss: 1.3553
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8621 - loss: 0.4387 - val_accuracy: 0.4217 - val_loss: 1.6273
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8982 - loss: 0.3278 - val_accuracy: 0.4502 - val_loss: 1.7719
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9272 - loss: 0.2300 - val_accuracy: 0.4502 - val_loss: 1.9627
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9447 - loss: 0.1983 - val_accuracy: 0

In [40]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [5000, 10000, 20000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense)
  # -----------------------------
  for EMBED_DIM in [32]:

    model = Sequential([
        Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\n===== TRAINING CONFIGURATION =====")
    print(f"MAX_VOCAB: {MAX_VOCAB}")
    print(f"MAX_LEN: {MAX_LEN}")
    print(f"Embedding dim: {EMBED_DIM}")
    print(f"Epochs: 20")
    print(f"Batch size: 16")
    print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
    print(f"Number of classes: {NUM_CLASSES}")
    model.build(input_shape=(None, MAX_LEN))
    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_pad,
        y_train_enc,
        validation_data=(X_val_pad, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_pad)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report:")
    print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_31 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_139 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_140 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_141 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_33 (Embedding)        │ (None, 100, 32)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_26 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_145 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_146 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_147 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 743,653 (2.84 MB)

 Trainable params: 743,653 (2.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.3472 - loss: 1.4515 - val_accuracy: 0.4146 - val_loss: 1.3165
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5226 - loss: 1.1864 - val_accuracy: 0.4662 - val_loss: 1.2553
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8716 - loss: 0.5012 - val_accuracy: 0.4395 - val_loss: 1.4763
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9398 - loss: 0.2127 - val_accuracy: 0.4555 - val_loss: 1.6125
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9530 - loss: 0.1498 - val_accuracy: 0.4573 - val_loss: 1.6798
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9588 - loss: 0.1258 - val_accuracy: 0.4502 - val_loss: 1.7469
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9639 - loss: 0.0996 - val_accuracy: 0.4502 - val_loss: 1.7183
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          pre

In [41]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [5000, 10000, 20000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.2, 0.3, 0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        print("\n===== TRAINING CONFIGURATION =====")
        print(f"MAX_VOCAB: {MAX_VOCAB}")
        print(f"MAX_LEN: {MAX_LEN}")
        print(f"Embedding dim: {EMBED_DIM}")
        print(f"Epochs: 20")
        print(f"Batch size: 16")
        print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
        print(f"Number of classes: {NUM_CLASSES}")
        print(f"Dropout rate: {DROPOUT_RATE}")
        model.build(input_shape=(None, MAX_LEN))
        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=16,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 5000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.2


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_46"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_34 (Embedding)        │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_148 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_56 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_149 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_150 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 263,653 (1.01 MB)

...

 Total params: 743,653 (2.84 MB)

 Trainable params: 743,653 (2.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.3035 - loss: 1.5002 - val_accuracy: 0.4306 - val_loss: 1.3342
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4139 - loss: 1.3340 - val_accuracy: 0.4555 - val_loss: 1.2760
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5679 - loss: 1.0998 - val_accuracy: 0.4733 - val_loss: 1.2779
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7053 - loss: 0.7745 - val_accuracy: 0.4822 - val_loss: 1.3051
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8337 - loss: 0.4770 - val_accuracy: 0.4804 - val_loss: 1.4723
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9098 - loss: 0.3138 - val_accuracy: 0.4626 - val_loss: 1.6294
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9241 - loss: 0.2442 - val_accuracy: 0.4715 - val_loss: 1.7601
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Max vocab:20000
Dropout rate:0.4

Test Classification Rep

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        print("\n===== TRAINING CONFIGURATION =====")
        print(f"MAX_VOCAB: {MAX_VOCAB}")
        print(f"MAX_LEN: {MAX_LEN}")
        print(f"Embedding dim: {EMBED_DIM}")
        print(f"Epochs: 20")
        print(f"Batch size: 16")
        print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
        print(f"Number of classes: {NUM_CLASSES}")
        print(f"Dropout rate: {DROPOUT_RATE}")
        model.build(input_shape=(None, MAX_LEN))
        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=16,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.4


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_55"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_43 (Embedding)        │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_36 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_175 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_74 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_176 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_75 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_177 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,653 (1.62 MB)

 Trainable params: 423,653 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.3418 - loss: 1.4620 - val_accuracy: 0.4146 - val_loss: 1.3445
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4175 - loss: 1.3425 - val_accuracy: 0.4591 - val_loss: 1.2699
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5391 - loss: 1.1467 - val_accuracy: 0.4573 - val_loss: 1.2215
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6790 - loss: 0.8439 - val_accuracy: 0.4698 - val_loss: 1.2671
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7910 - loss: 0.6182 - val_accuracy: 0.4484 - val_loss: 1.4022
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8628 - loss: 0.4357 - val_accuracy: 0.4644 - val_loss: 1.6156
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9186 - loss: 0.2917 - val_accuracy: 0.4537 - val_loss: 1.8080
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9250 - loss: 0.2432 - val_accuracy:

In [43]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
    MAX_LEN = 100

    tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq   = tokenizer.texts_to_sequences(X_val)
    X_test_seq  = tokenizer.texts_to_sequences(X_test)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
    X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
    X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

    # -----------------------------
    # Neural Network (Embedding → Flatten → Dense with Dropout + L2)
    # -----------------------------
    for EMBED_DIM in [32]:
        for DROPOUT_RATE in [0.4]:
            L2_REG = 1e-4  # L2 regularization factor

            model = Sequential([
                Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
                Flatten(),
                Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
                Dropout(DROPOUT_RATE),
                Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
                Dropout(DROPOUT_RATE),
                Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
            ])

            model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )

            print("\n===== TRAINING CONFIGURATION =====")
            print(f"MAX_VOCAB: {MAX_VOCAB}")
            print(f"MAX_LEN: {MAX_LEN}")
            print(f"Embedding dim: {EMBED_DIM}")
            print(f"Epochs: 20")
            print(f"Batch size: 16")
            print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
            print(f"Number of classes: {NUM_CLASSES}")
            print(f"Dropout rate: {DROPOUT_RATE}")
            print(f"L2 regularization factor: {L2_REG}")
            model.build(input_shape=(None, MAX_LEN))
            model.summary()

            # -----------------------------
            # EarlyStopping callback
            # -----------------------------
            early_stop = EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            )

            # -----------------------------
            # Train
            # -----------------------------
            model.fit(
                X_train_pad,
                y_train_enc,
                validation_data=(X_val_pad, y_val_enc),
                epochs=20,
                batch_size=16,
                verbose=1,
                callbacks=[early_stop]
            )

            # -----------------------------
            # Evaluate on test set
            # -----------------------------
            test_preds = model.predict(X_test_pad)
            test_preds = np.argmax(test_preds, axis=1)
            decoded_preds = label_encoder.inverse_transform(test_preds)

            print(f"Max vocab: {MAX_VOCAB}")
            print(f"Dropout rate: {DROPOUT_RATE}")
            print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
            print(classification_report(y_test, decoded_preds))



===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.4
L2 regularization factor: 0.0001


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_56"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_44 (Embedding)        │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_37 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_178 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_179 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_77 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_180 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,653 (1.62 MB)

 Trainable params: 423,653 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3226 - loss: 1.4820 - val_accuracy: 0.4181 - val_loss: 1.3521
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4356 - loss: 1.3352 - val_accuracy: 0.4431 - val_loss: 1.2941
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5372 - loss: 1.1667 - val_accuracy: 0.4626 - val_loss: 1.2624
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7106 - loss: 0.8167 - val_accuracy: 0.4698 - val_loss: 1.3431
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8429 - loss: 0.5214 - val_accuracy: 0.4858 - val_loss: 1.5719
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9109 - loss: 0.3442 - val_accuracy: 0.4680 - val_loss: 1.6466
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9344 - loss: 0.2801 - val_accuracy: 0.4733 - val_loss: 1.8566
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9377 - loss: 0.2340 - val_accuracy:

In [44]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        print("\n===== TRAINING CONFIGURATION =====")
        print(f"MAX_VOCAB: {MAX_VOCAB}")
        print(f"MAX_LEN: {MAX_LEN}")
        print(f"Embedding dim: {EMBED_DIM}")
        print(f"Epochs: 20")
        print(f"Batch size: 8")
        print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
        print(f"Number of classes: {NUM_CLASSES}")
        print(f"Dropout rate: {DROPOUT_RATE}")
        model.build(input_shape=(None, MAX_LEN))
        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=8,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 8
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.4


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_57"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_45 (Embedding)        │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_38 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_181 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_78 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_182 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_79 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_183 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,653 (1.62 MB)

 Trainable params: 423,653 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3351 - loss: 1.4654 - val_accuracy: 0.4199 - val_loss: 1.3215
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4407 - loss: 1.3222 - val_accuracy: 0.4484 - val_loss: 1.2572
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5954 - loss: 1.0469 - val_accuracy: 0.4715 - val_loss: 1.2583
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7721 - loss: 0.6600 - val_accuracy: 0.4591 - val_loss: 1.4350
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8777 - loss: 0.4003 - val_accuracy: 0.4520 - val_loss: 1.7147
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9168 - loss: 0.2637 - val_accuracy: 0.3612 - val_loss: 1.8802
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9456 - loss: 0.2305 - val_accuracy: 0.4324 - val_loss: 1.9426
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Max vocab:10000
Dropout rate:0.4

Test Classification Report (E

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Végső választott modell.
Később a dropout törölve lett, mert úgy jobb eredményt adott.

In [45]:
#Végső modell tesztelése új adathalmaz felbontásban


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Split dataset into train / val / test
# -----------------------------
# First split: train vs temp (val + test)


# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print(f"\nMax vocab: {MAX_VOCAB}")
print(f"Dropout rate: {DROPOUT_RATE}")
print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
print(classification_report(y_test, decoded_preds))



===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.4


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_58"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_46 (Embedding)        │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_39 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_184 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_80 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_185 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_81 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_186 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,653 (1.62 MB)

 Trainable params: 423,653 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.3494 - loss: 1.4712 - val_accuracy: 0.4181 - val_loss: 1.3255
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4291 - loss: 1.3227 - val_accuracy: 0.4520 - val_loss: 1.2744
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5524 - loss: 1.1148 - val_accuracy: 0.4626 - val_loss: 1.2564
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7331 - loss: 0.7358 - val_accuracy: 0.4893 - val_loss: 1.3346
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8418 - loss: 0.5054 - val_accuracy: 0.4733 - val_loss: 1.5081
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8984 - loss: 0.3563 - val_accuracy: 0.4644 - val_loss: 1.7455
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9184 - loss: 0.2773 - val_accuracy: 0.4591 - val_loss: 1.8442
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9427 - loss: 0.2156 - val_accuracy: 

In [46]:
#Lehetséges megoldás a kiegyenlítetlenségre
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

#Végső modell tesztelése új adathalmaz felbontásban


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np



# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n===== TRAINING CONFIGURATION =====")
print(f"MAX_VOCAB: {MAX_VOCAB}")
print(f"MAX_LEN: {MAX_LEN}")
print(f"Embedding dim: {EMBED_DIM}")
print(f"Epochs: 20")
print(f"Batch size: 16")
print(f"Learning rate: {model.optimizer.learning_rate.numpy()}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Dropout rate: {DROPOUT_RATE}")
model.build(input_shape=(None, MAX_LEN))
model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

y_train_int = label_encoder.transform(y_train)  # already done as y_train_enc before one-hot
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_int),
    y=y_train_int
)
class_weights_dict = dict(enumerate(class_weights))

# -----------------------------
# Train model with class weights
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop],
    class_weight=class_weights_dict   # <-- add here
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print(f"\nMax vocab: {MAX_VOCAB}")
print(f"Dropout rate: {DROPOUT_RATE}")
print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
print(classification_report(y_test, decoded_preds))


===== TRAINING CONFIGURATION =====
MAX_VOCAB: 10000
MAX_LEN: 100
Embedding dim: 32
Epochs: 20
Batch size: 16
Learning rate: 0.0010000000474974513
Number of classes: 5
Dropout rate: 0.4


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_59"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_47 (Embedding)        │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_40 (Flatten)            │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_187 (Dense)               │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_82 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_188 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_83 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_189 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,653 (1.62 MB)

 Trainable params: 423,653 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2952 - loss: 1.6141 - val_accuracy: 0.3310 - val_loss: 1.5254
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3253 - loss: 1.4753 - val_accuracy: 0.3950 - val_loss: 1.3800
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4439 - loss: 1.2449 - val_accuracy: 0.4342 - val_loss: 1.3801
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5281 - loss: 1.0419 - val_accuracy: 0.4751 - val_loss: 1.2536
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6625 - loss: 0.7400 - val_accuracy: 0.4448 - val_loss: 1.3544
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7532 - loss: 0.5580 - val_accuracy: 0.4146 - val_loss: 1.4606
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8320 - loss: 0.4296 - val_accuracy: 0.4164 - val_loss: 1.5533
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8578 - loss: 0.3680 - val_accuracy:

Accuracy romlott, de minden osztályra becsül a modell (végül nem lett használva)